In [1]:
from __future__ import print_function, division

import tensorflow as tf

print(tf.__version__)

import keras

import matplotlib.pyplot as plt

import sys

from sklearn import preprocessing

import time
from tqdm import tqdm

from imutils import paths
from numpy import *
import numpy as np

from matplotlib.colors import Normalize

import argparse
import imutils,sklearn
import os, cv2, re, random, shutil, imageio, pickle

%matplotlib inline  

1.3.0


/home/huiqy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]

def load_flower_data():
    # grab the list of images that we'll be describing
    print("[INFO] handling images...")
    TRAIN_ORIGINAL_DIR = '../train/'
    TRAIN_SUB_DIR = '../subsample/'
    TEST_DIR = '../test/'

    # use this for full dataset
    train_images_original = [TRAIN_ORIGINAL_DIR + i for i in os.listdir(TRAIN_ORIGINAL_DIR)]
    train_images_sub = [TRAIN_SUB_DIR + i for i in os.listdir(TRAIN_SUB_DIR)]

    train_images = train_images_original + train_images_sub

    #print (train_images)
    train_images.sort(key=natural_keys)
    print (len(train_images))

    # initialize the features matrix and labels list
    trainImage = []
    trainLabels = []

    # loop over the input images
    for (i, imagePath) in enumerate(train_images):
        # extract the class label
        # get the labels from the name of the images by extract the string before "_"
        label = imagePath.split(os.path.sep)[-1].split("_")[0]

        # read and resize image
        img = cv2.imread(imagePath)
        img = cv2.resize(img, (128,128))

        # add the messages we got to features and labels matricies
        trainImage.append(img)
        trainLabels.append(label)

        # show an update every 100 images until the last image
#         if i > 0 and ((i + 1) % 1000 == 0 or i == len(train_images) - 1):
#             print("[INFO] processed {}/{}".format(i + 1, len(train_images)))

    trainImage = np.array(trainImage,dtype = float32)
    trainLabels = np.array(trainLabels)
    print (trainImage.shape)
    trainImage = trainImage.astype(np.float32) / 255
    
    le = preprocessing.LabelEncoder()
    le.fit(trainLabels)
    list(le.classes_)
    trainLabels = le.transform(trainLabels) 
    
    return trainImage, trainLabels

In [3]:
trainImage,trainLabels = load_flower_data()

nb_classes = 2

# Convert class vectors to binary class matrices.
trainLabels = keras.utils.to_categorical(trainLabels, nb_classes)
print (trainLabels)
np.save('./trainImage.npy', trainImage)
np.save('./trainLabels.npy', trainLabels)

print("[INFO] trainImage matrix: {:.2f}MB".format(
    (trainImage.nbytes) / (1024 * 1000.0)))
print("[INFO] trainLabels matrix: {:.4f}MB".format(
    (trainLabels.nbytes) / (1024 * 1000.0)))


[INFO] handling images...
6006
(6006, 128, 128, 3)
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[INFO] trainImage matrix: 1153.15MB
[INFO] trainLabels matrix: 0.0938MB


In [4]:
def leaky_relu( X, leak=0.2):
    f1 = 0.5 * (1 + leak)
    f2 = 0.5 * (1 - leak)
    return f1 * X + f2 * tf.abs(X)   
    

In [5]:
class cDCGAN(object):
    def __init__(self, sess, epoch, batch_size, dataset_name, checkpoint_dir, result_dir, log_dir, 
                 test_dir, trainhist_dir):
        self.sess = sess
        self.dataset_name = dataset_name
        self.checkpoint_dir = checkpoint_dir
        self.result_dir = result_dir
        self.log_dir = log_dir
        self.test_dir = test_dir
        self.trainhist_dir = trainhist_dir
        self.epoch = epoch
        self.batch_size = batch_size
        self.model_name = "cDCGAN"     # name for checkpoint
        
        self.classname = ['Iris', 'Pansy']

        # parameters
        self.input_height = 128
        self.input_width = 128
        self.output_height = 128
        self.output_width = 128

        self.z_dim = 100         # dimension of noise-vector
        self.c_dim = 3  # color dimension
        self.nb_class = 2

        # train
        #设置一个全局的计数器
        self.global_step = tf.Variable(0, trainable=False)
        self.lr = tf.train.exponential_decay( 0.0002,
                                             global_step=self.global_step,
                                             decay_steps=20,
                                             decay_rate=0.9,
                                             staircase=True)
        self.beta1 = 0.5
        #max model to keep saving
        self.max_to_keep = 300

        # test
        #visual results
        self.onehot = np.eye(self.nb_class)
        self.sample_num = 100  # number of generated images to be saved
        self.visual_team = self.batch_size // self.nb_class  # number of generated images to be saved

        #load_flower_data
        self.data_x = np.load('./trainImage.npy')
        self.data_y= np.load('./trainLabels.npy')
        
        #记录
        self.train_hist = {}
        self.train_hist['D_losses'] = []
        self.train_hist['G_losses'] = []
        self.train_hist['learning_rate'] = []
        self.train_hist['per_epoch_ptimes'] = []
        self.train_hist['total_ptime'] = []
        
        # get number of batches for a single epoch
        self.num_batches = len(self.data_x) // self.batch_size

    def discriminator(self, x, y_fill, is_training=True, reuse=False):
        with tf.variable_scope("discriminator", reuse=reuse):
             
            #初始化参数
            W = tf.truncated_normal_initializer(mean=0.0, stddev=0.02)
            b = tf.constant_initializer(0.0)
        
            print("D:x",x.get_shape()) # 32, 32, 3 = 3072
            print("D:y_fill",y_fill.get_shape())
            # concat layer
            
            #卷积核为4*4 输出维度为32
            net1_1 = tf.layers.conv2d(x, 32, [4, 4], strides=(2, 2), padding='same', 
                                     kernel_initializer=W, bias_initializer=b)
            net1_1 = leaky_relu(net1_1, 0.2)
            
            print("D:",net1_1.get_shape())
            
            net1_2 = tf.layers.conv2d(y_fill, 32, [4, 4], strides=(2, 2), padding='same', 
                         kernel_initializer=W, bias_initializer=b)
            net1_2 = leaky_relu(net1_2, 0.2)
            print("D:",net1_2.get_shape())
    
            #把数据和标签进行连接
            net = tf.concat([net1_1, net1_2], 3)
            print("D:",net.get_shape())
            
            #卷积核为4*4 输出维度为128        
            net = tf.layers.conv2d(net, 64, [4, 4], strides=(2, 2), padding='same', 
                                     kernel_initializer=W, bias_initializer=b)
            net = tf.layers.batch_normalization(net, training=is_training)#batch norm
            net = leaky_relu(net, 0.2)
            print("D:",net.get_shape())
    
            #卷积核为4*4 输出维度为128 
            net = tf.layers.conv2d(net, 128, [4, 4], strides=(2, 2), padding='same', 
                                 kernel_initializer=W, bias_initializer=b)
            net = tf.layers.batch_normalization(net, training=is_training)#batch norm
            net = leaky_relu(net, 0.2)
            print("D:",net.get_shape())
            
            #卷积核为4*4 输出维度为256
            net = tf.layers.conv2d(net, 256, [4, 4], strides=(2, 2), padding='same', 
                     kernel_initializer=W, bias_initializer=b)
            net = tf.layers.batch_normalization(net, training=is_training)#batch norm
            net = leaky_relu(net, 0.2)
            print("D:",net.get_shape())
            
            #卷积核为4*4 输出维度为512
            net = tf.layers.conv2d(net, 512, [4, 4], strides=(2, 2), padding='same', 
                 kernel_initializer=W, bias_initializer=b)
            net = tf.layers.batch_normalization(net, training=is_training)#batch norm
            net = leaky_relu(net, 0.2)
            print("D:",net.get_shape())
            
            #output
            out_logit = tf.layers.conv2d(net, 1, [4, 4], strides=(1, 1), padding='valid', 
                 kernel_initializer=W, bias_initializer=b)
            print("D:out_logit",out_logit.get_shape())
            
            out = tf.nn.sigmoid(out_logit)
            print("D:out",out.get_shape())
            print("------------------------")    

            return out, out_logit, net

    def generator(self, z, y_label, is_training=True, reuse=False):
        with tf.variable_scope("generator", reuse=reuse):
            
            #初始化参数
            W = tf.truncated_normal_initializer(mean=0.0, stddev=0.02)
            b = tf.constant_initializer(0.0)
            
            print("G:z",z.get_shape())
            print("G:y_label",y_label.get_shape())
            
            #反卷积,卷积核大小为4*4，输出维度256
            net1_1 = tf.layers.conv2d_transpose(z, 256, [4, 4], strides=(1, 1), padding='valid',
                                               kernel_initializer=W, bias_initializer=b)
            net1_1 = leaky_relu(net1_1, 0.2)
            print("G:",net1_1.get_shape())
            
            #反卷积,卷积核大小为4*4，输出维度256
            net1_2 = tf.layers.conv2d_transpose(y_label, 256, [4, 4], strides=(1, 1), padding='valid',
                                               kernel_initializer=W, bias_initializer=b)
            net1_2 = leaky_relu(net1_2, 0.2)
            print("G:",net1_2.get_shape())
            
            #把数据和标签进行连接
            # concat layer
            net = tf.concat([net1_1, net1_2], 3)
            print("G:",net.get_shape())
            
            #反卷积,卷积核大小为4*4，输出维度512
            net = tf.layers.conv2d_transpose(net, 512, [4, 4], strides=(2, 2), padding='same',
                                             kernel_initializer=W, bias_initializer=b)
            net = tf.layers.batch_normalization(net, training=is_training)#batch norm
            net = leaky_relu(net, 0.2)
            print("G:",net.get_shape())
            
            #反卷积,卷积核大小为4*4，输出维度128
            net = tf.layers.conv2d_transpose(net, 128, [4, 4], strides=(2, 2), padding='same',
                                             kernel_initializer=W, bias_initializer=b)
            net = tf.layers.batch_normalization(net, training=is_training)#batch norm
            net = leaky_relu(net, 0.2)
            print("G:",net.get_shape())
            
            #反卷积,卷积核大小为4*4，输出维度64
            net = tf.layers.conv2d_transpose(net, 64, [4, 4], strides=(2, 2), padding='same',
                                             kernel_initializer=W, bias_initializer=b)
            net = tf.layers.batch_normalization(net, training=is_training)#batch norm
            net = leaky_relu(net, 0.2)
            print("G:",net.get_shape())
            
            #反卷积,卷积核大小为4*4，输出维度32
            net = tf.layers.conv2d_transpose(net, 32, [4, 4], strides=(2, 2), padding='same',
                                             kernel_initializer=W, bias_initializer=b)
            net = tf.layers.batch_normalization(net, training=is_training)#batch norm
            net = leaky_relu(net, 0.2)
            print("G:",net.get_shape())
            
            #反卷积,卷积核大小为4*4，输出维度3
            net = tf.layers.conv2d_transpose(net, 3, [4, 4], strides=(2, 2), padding='same',
                                 kernel_initializer=W, bias_initializer=b)
            out = tf.nn.tanh(net)
            print("G:",out.get_shape())
            print("------------------------")

            return out

    def build_model(self):
        #parameters
        image_dims = [self.input_height, self.input_width, self.c_dim]

        """ Graph Input """
        # images
        self.x = tf.placeholder(tf.float32, shape=(self.batch_size,self.input_height, self.input_width, self.c_dim), 
                                name='real_images')

        # noises
        self.z = tf.placeholder(tf.float32, shape=(self.batch_size, 1, 1, self.z_dim), name='z')
        
        self.y_label = tf.placeholder(tf.float32, shape=(self.batch_size, 1, 1, self.nb_class), name='y_label')
        self.y_fill = tf.placeholder(tf.float32, shape=(self.batch_size, self.output_width, 
                                                        self.output_height, self.nb_class), name='y_fill')
        
        self.add_global = self.global_step.assign_add(1)

        """ Loss Function """

        # output of D for real images
        D_real, D_real_logits, _ = self.discriminator(self.x, self.y_fill, is_training=True, reuse=False)
            
        #networks : generator
        G = self.generator(self.z, self.y_label, is_training=True, reuse=False)

        # output of D for fake images
        D_fake, D_fake_logits, _ = self.discriminator(G, self.y_fill, is_training=True, reuse=True)

        # get loss for discriminator
        self.d_loss_real = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real_logits, labels=tf.ones_like(D_real)))
        self.d_loss_fake = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.zeros_like(D_fake)))

        self.d_loss = self.d_loss_real + self.d_loss_fake
        
        # get loss for generator
        self.g_loss = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.ones_like(D_fake)))
        
        """ Training """
        # divide trainable variables into a group for D and a group for G
        tf_vars = tf.trainable_variables()
        d_vars = [var for var in tf_vars if var.name.startswith('discriminator')]
        g_vars = [var for var in tf_vars if var.name.startswith('generator')]

        # optimizers
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            self.d_optim = tf.train.AdamOptimizer(self.lr, beta1=self.beta1).minimize(self.d_loss, var_list=d_vars)
            self.g_optim = tf.train.AdamOptimizer(self.lr, beta1=self.beta1).minimize(self.g_loss, var_list=g_vars)
        
        """" Testing """
        # for test
        self.fake_images = self.generator(self.z, self.y_label, is_training=False, reuse=True)

        """ Summary """
        d_loss_real_sum = tf.summary.scalar("d_loss_real", self.d_loss_real)
        d_loss_fake_sum = tf.summary.scalar("d_loss_fake", self.d_loss_fake)
        d_loss_sum = tf.summary.scalar("d_loss", self.d_loss)
        g_loss_sum = tf.summary.scalar("g_loss", self.g_loss)

        # final summary operations
        self.g_sum = tf.summary.merge([d_loss_fake_sum, g_loss_sum])
        self.d_sum = tf.summary.merge([d_loss_real_sum, d_loss_sum])

    def train(self):

        # initialize all variables
        tf.global_variables_initializer().run()

        # graph inputs for visualize training results
        self.sample_z = np.random.normal(0, 1, (self.batch_size, 1, 1, 100)).astype(np.float32)

        self.test_images = self.data_x[0:self.batch_size]

        # saver to save model
        self.saver = tf.train.Saver(max_to_keep= self.max_to_keep)

        # summary writer
        self.writer = tf.summary.FileWriter(self.log_dir + '/' + self.model_name, self.sess.graph)

        # restore check-point if it exits
        could_load, checkpoint_epoch = self.load(self.checkpoint_dir)
        if could_load:
            start_epoch = (int)(checkpoint_epoch) + 1
            counter = 1
            f = open(self.trainhist_dir + '/' + 'train_hist.pkl', 'rb') 
            self.train_hist = pickle.load(f)
            f.close()
            print(" [*] Load SUCCESS")
            print(" [!] START_EPOCH is ", start_epoch)
        else:
            start_epoch = 1
            counter = 1
            print(" [!] Load failed...")

        # loop for epoch
        start_time = time.time()
        for epoch_loop in range(start_epoch, self.epoch + 1):

            G_losses = []
            D_losses = []
    
            epoch_start_time = time.time()
            shuffle_idxs = random.sample(range(0, self.data_x.shape[0]), self.data_x.shape[0])
            shuffled_set = self.data_x[shuffle_idxs]
            shuffled_label = self.data_y[shuffle_idxs]
    
            # get batch data
            for idx in range(self.num_batches):
                batch_x = shuffled_set[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_y_label = shuffled_label[idx*self.batch_size:(idx+1)*self.batch_size].reshape(
                                        [self.batch_size, 1, 1, self.nb_class])
                batch_y_fill = batch_y_label * np.ones([self.batch_size, self.output_height, self.output_width, self.nb_class])
                batch_z = np.random.normal(0, 1, (self.batch_size, 1, 1, 100)).astype(np.float32)
                

                # update D network
                _, summary_str, d_loss = self.sess.run([self.d_optim, self.d_sum, self.d_loss],
                                               feed_dict={self.x: batch_x,
                                                          self.z: batch_z,
                                                          self.y_fill: batch_y_fill,
                                                          self.y_label: batch_y_label}
                                                      )
                self.writer.add_summary(summary_str, counter)

                # update G network
                batch_y = np.random.randint(0, self.nb_class, (self.batch_size, 1)) # <=  <
                batch_y_label = self.onehot[batch_y.astype(np.int32)].reshape([self.batch_size, 1, 1, self.nb_class])
                batch_y_fill  = batch_y_label * np.ones([self.batch_size, self.input_height, self.input_width, self.nb_class])

                # update G once
                _, summary_str, g_loss = self.sess.run([self.g_optim, self.g_sum, self.g_loss],
                                                          feed_dict={self.x: batch_x,
                                                                     self.z: batch_z, 
                                                                     self.y_fill: batch_y_fill,
                                                                     self.y_label: batch_y_label})
                self.writer.add_summary(summary_str, counter)
                
                D_losses.append(d_loss)
                G_losses.append(g_loss)

                # display training status
                counter += 1
                print("Epoch: [%2d] [%4d/%4d] time: %4.4f, d_loss: %.8f, g_loss: %.8f" % (epoch_loop, idx, self.num_batches, time.time() - start_time, d_loss, g_loss))

            # After an epoch
            #update learning rate
            _, rate = sess.run([self.add_global, self.lr])
            epoch_end_time = time.time()
            per_epoch_ptime = epoch_end_time - epoch_start_time
            
            print('[%d/%d] - ptime: %.4f loss_d: %.8f, loss_g: %.8f, lf:  %.8f' % (epoch_loop, self.epoch, per_epoch_ptime, 
                                                                        np.mean(D_losses), np.mean(G_losses), rate))
            self.train_hist['D_losses'].append(np.mean(D_losses))
            self.train_hist['G_losses'].append(np.mean(G_losses))
            self.train_hist['learning_rate'].append(rate)
            self.train_hist['per_epoch_ptimes'].append(per_epoch_ptime)
            
            # save model
            self.save(self.checkpoint_dir, epoch_loop)

            # show temporal results
            self.visualize_epoch_results(epoch_loop)
            
            # save trainhist for the initial train
            f = open(self.trainhist_dir + '/' + 'train_hist.pkl', 'wb') 
            pickle.dump(self.train_hist, f)
            f.close()
            
            self.show_train_hist(self.train_hist, save=True, path= self.trainhist_dir + '/' + 'train_hist.png')

        end_time = time.time()
        total_ptime = end_time - start_time
        self.train_hist['total_ptime'].append(total_ptime)
        print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(self.train_hist['per_epoch_ptimes']), 
                                                                          self.epoch, total_ptime))
        print(" [*] Training finished!")
        
    def visualize_img_to_gif(self, path, size, fps = 5):
        print(" [*] Generation Animation GIF start!")
        path_images = [path + '/'+ i for i in os.listdir(path)]
        path_images.sort(key=natural_keys)
        print(" [*] Reading " + str(len(path_images)) + " " +  str(size) + " images start!")
        images = []
        for p in tqdm(path_images):
            img = imageio.imread(p)
            img = cv2.resize(img, size)
            images.append(img)
        print(" [*] Reading " + str(len(path_images)) + " " +  str(size) + " images finished!")
        print(" [*] Generation Animation "+ str(size) + " GIF start! FPS=", fps)
        path_gif = path + '/' + self.model_name + '_epoch%03d' % self.epoch + '_' + self.dataset_name + 'generation_animation.gif'
        imageio.mimsave(path_gif, images, fps = fps)
        print(" [*] Generation Animation "+ str(size) + " GIF finished! FPS=", fps)

    def visualize_test_image(self, epoch, num, times):
        tot_num_samples = num #  100
        path_model = self.checkpoint_dir + '/' + self.model_dir + '/' + self.model_name + '/'+ self.model_name +'.model-'+ str(epoch)
        
        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)
            
        """ random condition, random noise """
        for j in range(times):
            noise = np.random.normal(0, 1, (self.visual_team, 1, 1, self.z_dim))
            fixed_noise = noise 
            fixed_label = np.zeros((self.visual_team , 1))
            #用于显示25组图像
            for i in range(self.nb_class - 1):
                fixed_noise = np.concatenate([fixed_noise, noise], 0)
                temp_label = np.ones((self.visual_team, 1)) + i
                fixed_label = np.concatenate([fixed_label, temp_label], 0)
            fixed_label = self.onehot[fixed_label.astype(np.int32)].reshape((self.batch_size, 1, 1, self.nb_class))
            samples = self.sess.run(self.fake_images, feed_dict={self.z: fixed_noise, self.y_label: fixed_label})
            self.save_cv2_img(samples[:tot_num_samples, :, :, :], tot_num_samples, epoch , j) 
        print(" [*] Finished generating Epoch:", epoch, ",",times*num,"new images!")
        
                                    
    def save_cv2_img(self, images, num, epoch, times):
    # revice image data // M*N*3 // RGB float32 : value must set between 0. with 1.
        for idx in range(num):
            classname = self.classname[idx // self.visual_team]
            img_save = images[idx:idx+1, :, :, :][0, ...]*255
            path = self.test_dir + '/' + classname + '_epoch%03d' % epoch + '_t%02d' % times+ '_%03d' % idx + '.png'
            cv2.imwrite( path, img_save)
              
    def show_all_variables(self):
        model_vars = tf.trainable_variables()
        tf.contrib.slim.model_analyzer.analyze_vars(model_vars, print_info=True)
        
    def save_matplot_img(self, images, size, image_path):
        # revice image data // M*N*3 // RGB float32 : value must set between 0. with 1.
        for idx in range(100):
            vMin = np.amin(images[idx])
            vMax = np.amax(images[idx])
            img_arr = images[idx].reshape(128*128*3,1) # flatten
            for i, v in enumerate(img_arr):
                img_arr[i] = (v-vMin)/(vMax-vMin)
            img_arr = img_arr.reshape(128,128,3) # M*N*3

            # matplot display
            plt.subplot(10,10,idx+1),plt.imshow(cv2.cvtColor(img_arr, cv2.COLOR_BGR2RGB), interpolation='nearest')
            #plt.title("pred.:{}".format(np.argmax(self.data_y[0]),fontsize=10))
            plt.axis("off")

        plt.savefig(image_path, dpi = 400)
        plt.close()
        

    @property
    def model_dir(self):
        return "{}_{}_{}_{}".format(
            self.dataset_name, self.batch_size,
            self.output_height, self.output_width)

    def save(self, checkpoint_dir, step):
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_dir, self.model_name)

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        self.saver.save(self.sess,os.path.join(checkpoint_dir, self.model_name+'.model'), global_step=step)

    def load(self, checkpoint_dir):
        import re
        print(" [*] Reading checkpoints...")
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_dir, self.model_name)

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
            epoch = int(next(re.finditer("(\d+)(?!.*\d)",ckpt_name)).group(0))
            print(" [*] Success to read [{}], epoch [{}]".format(ckpt_name,epoch))
            return True, epoch
        else:
            print(" [*] Failed to find a checkpoint")
            return False, 0
        
    def show_train_hist(self, hist, show = False, save = False, path = 'Train_hist.png'):
        x = range(1, len(hist['D_losses'])+1)

        y1 = hist['D_losses']
        y2 = hist['G_losses']

        plt.plot(x, y1, label='D_loss')
        plt.plot(x, y2, label='G_loss')

        plt.xlabel('Epoch')
        plt.ylabel('Loss')

        plt.legend(loc=4)
        plt.grid(True)
        plt.tight_layout()

        if save:
            plt.savefig(path, dpi = 400)

        if show:
            plt.show()
        else:
            plt.close()
        
    def visualize_epoch_results(self, epoch):
        tot_num_samples = min(self.sample_num, self.batch_size) # 100
        image_frame_dim = int(np.floor(np.sqrt(tot_num_samples))) # 10
        
        """ random condition, random noise """
            
        noise = np.random.normal(0, 1, (self.visual_team, 1, 1, self.z_dim))
        fixed_noise = noise 
        fixed_label = np.zeros((self.visual_team , 1))
        
        #用于显示25组图像
        for i in range(self.nb_class - 1):
            fixed_noise = np.concatenate([fixed_noise, noise], 0)
            temp_label = np.ones((self.visual_team, 1)) + i
            fixed_label = np.concatenate([fixed_label, temp_label], 0)
        fixed_label = self.onehot[fixed_label.astype(np.int32)].reshape((self.batch_size, 1, 1, self.nb_class))
    
        samples = self.sess.run(self.fake_images, feed_dict={self.z: fixed_noise, self.y_label: fixed_label})

        self.save_matplot_img(samples[:image_frame_dim * image_frame_dim, :, :, :], [image_frame_dim, image_frame_dim],
                    self.result_dir + '/' + self.model_name + '_epoch%03d' % epoch + '_' + self.dataset_name +'.png')

   

In [6]:
gan_type = 'cDCGAN'
dataset = 'Class2_Sample3_test1_1_1000'
epoch = 200
batch_size = 100
checkpoint_dir = 'checkpoint'
result_dir = 'results'
log_dir = 'logs'
test_dir = 'image_gan'
trainhist_dir = 'trainhist'

if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

# --result_dir
if not os.path.exists(result_dir):
    os.makedirs(result_dir)

# --result_dir
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
# --test_dir
if not os.path.exists(test_dir):
    os.makedirs(test_dir)
    
# --trainhist_dir
if not os.path.exists(trainhist_dir):
    os.makedirs(trainhist_dir)

# open session
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    
    # declare instance for GAN
    cDCGAN = cDCGAN(sess, epoch=epoch, batch_size=batch_size, dataset_name=dataset, checkpoint_dir=checkpoint_dir, 
                    result_dir=result_dir, log_dir=log_dir, test_dir=test_dir, trainhist_dir=trainhist_dir)

    # build graph
    cDCGAN.build_model()

    # show network architecture
    cDCGAN.show_all_variables()

    # launch the graph in a session
    cDCGAN.train()
    
    #visualize_img_to_gif
    cDCGAN.visualize_img_to_gif(path=result_dir, size=(900,600), fps=5)

    vepoch = random.sample(range(100,170), 60)

    for e in tqdm(vepoch):
        cDCGAN.visualize_test_image( e, 100, 1)
        
sess.close()
        

D:x (100, 128, 128, 3)
D:y_fill (100, 128, 128, 2)
D: (100, 64, 64, 32)
D: (100, 64, 64, 32)
D: (100, 64, 64, 64)
D: (100, 32, 32, 64)
D: (100, 16, 16, 128)
D: (100, 8, 8, 256)
D: (100, 4, 4, 512)
D:out_logit (100, 1, 1, 1)
D:out (100, 1, 1, 1)
------------------------
G:z (100, 1, 1, 100)
G:y_label (100, 1, 1, 2)
G: (100, 4, 4, 256)
G: (100, 4, 4, 256)
G: (100, 4, 4, 512)
G: (100, 8, 8, 512)
G: (100, 16, 16, 128)
G: (100, 32, 32, 64)
G: (100, 64, 64, 32)
G: (100, 128, 128, 3)
------------------------
D:x (100, 128, 128, 3)
D:y_fill (100, 128, 128, 2)
D: (100, 64, 64, 32)
D: (100, 64, 64, 32)
D: (100, 64, 64, 64)
D: (100, 32, 32, 64)
D: (100, 16, 16, 128)
D: (100, 8, 8, 256)
D: (100, 4, 4, 512)
D:out_logit (100, 1, 1, 1)
D:out (100, 1, 1, 1)
------------------------
G:z (100, 1, 1, 100)
G:y_label (100, 1, 1, 2)
G: (100, 4, 4, 256)
G: (100, 4, 4, 256)
G: (100, 4, 4, 512)
G: (100, 8, 8, 512)
G: (100, 16, 16, 128)
G: (100, 32, 32, 64)
G: (100, 64, 64, 32)
G: (100, 128, 128, 3)
-----------

Epoch: [194] [  45/  60] time: 24.4717, d_loss: 0.06856532, g_loss: 4.84311485
Epoch: [194] [  46/  60] time: 24.9368, d_loss: 0.08855126, g_loss: 4.02815771
Epoch: [194] [  47/  60] time: 25.4012, d_loss: 0.04519202, g_loss: 3.46830058
Epoch: [194] [  48/  60] time: 25.8613, d_loss: 0.12589398, g_loss: 5.03662682
Epoch: [194] [  49/  60] time: 26.3328, d_loss: 0.06118522, g_loss: 5.06762123
Epoch: [194] [  50/  60] time: 26.8081, d_loss: 0.04918308, g_loss: 5.49938679
Epoch: [194] [  51/  60] time: 27.2729, d_loss: 0.05498641, g_loss: 4.17603493
Epoch: [194] [  52/  60] time: 27.7489, d_loss: 0.06353293, g_loss: 4.38031006
Epoch: [194] [  53/  60] time: 28.2118, d_loss: 0.03952925, g_loss: 4.56011581
Epoch: [194] [  54/  60] time: 28.6892, d_loss: 0.05028833, g_loss: 5.20901918
Epoch: [194] [  55/  60] time: 29.1570, d_loss: 0.03678952, g_loss: 4.57510710
Epoch: [194] [  56/  60] time: 29.6213, d_loss: 0.05215177, g_loss: 4.83979177
Epoch: [194] [  57/  60] time: 30.0884, d_loss: 0.03

Epoch: [196] [  27/  60] time: 104.2935, d_loss: 0.04975280, g_loss: 4.45365429
Epoch: [196] [  28/  60] time: 104.7542, d_loss: 0.07150847, g_loss: 5.02605724
Epoch: [196] [  29/  60] time: 105.2366, d_loss: 0.04799389, g_loss: 5.33975697
Epoch: [196] [  30/  60] time: 105.7016, d_loss: 0.05783553, g_loss: 4.20861197
Epoch: [196] [  31/  60] time: 106.1661, d_loss: 0.03800731, g_loss: 4.07607841
Epoch: [196] [  32/  60] time: 106.6291, d_loss: 0.05017956, g_loss: 4.39074898
Epoch: [196] [  33/  60] time: 107.0973, d_loss: 0.09258578, g_loss: 5.07843113
Epoch: [196] [  34/  60] time: 107.5854, d_loss: 0.08011502, g_loss: 5.35803032
Epoch: [196] [  35/  60] time: 108.0509, d_loss: 0.02707359, g_loss: 4.37766504
Epoch: [196] [  36/  60] time: 108.5152, d_loss: 0.05904130, g_loss: 4.59938622
Epoch: [196] [  37/  60] time: 108.9808, d_loss: 0.03780190, g_loss: 4.90116072
Epoch: [196] [  38/  60] time: 109.4431, d_loss: 0.05969931, g_loss: 4.93968439
Epoch: [196] [  39/  60] time: 109.9135,

Epoch: [198] [   8/  60] time: 183.2754, d_loss: 0.08473474, g_loss: 4.84861565
Epoch: [198] [   9/  60] time: 183.7528, d_loss: 0.03684603, g_loss: 5.32169962
Epoch: [198] [  10/  60] time: 184.2203, d_loss: 0.05784702, g_loss: 4.72110176
Epoch: [198] [  11/  60] time: 184.6942, d_loss: 0.03909322, g_loss: 4.32872009
Epoch: [198] [  12/  60] time: 185.1612, d_loss: 0.08225904, g_loss: 3.91064715
Epoch: [198] [  13/  60] time: 185.6243, d_loss: 0.05837026, g_loss: 4.37937450
Epoch: [198] [  14/  60] time: 186.0938, d_loss: 0.05259188, g_loss: 4.91894484
Epoch: [198] [  15/  60] time: 186.5607, d_loss: 0.03655646, g_loss: 4.60840893
Epoch: [198] [  16/  60] time: 187.0316, d_loss: 0.06903021, g_loss: 5.56140900
Epoch: [198] [  17/  60] time: 187.4984, d_loss: 0.05134577, g_loss: 4.52007341
Epoch: [198] [  18/  60] time: 187.9721, d_loss: 0.06101703, g_loss: 3.85306072
Epoch: [198] [  19/  60] time: 188.4598, d_loss: 0.04353867, g_loss: 4.42543030
Epoch: [198] [  20/  60] time: 188.9311,

Epoch: [199] [  50/  60] time: 247.9545, d_loss: 0.05602770, g_loss: 4.51212358
Epoch: [199] [  51/  60] time: 248.4579, d_loss: 0.04786793, g_loss: 4.58382750
Epoch: [199] [  52/  60] time: 248.9502, d_loss: 0.03434230, g_loss: 4.98009157
Epoch: [199] [  53/  60] time: 249.4519, d_loss: 0.06957927, g_loss: 4.45877218
Epoch: [199] [  54/  60] time: 249.9351, d_loss: 0.04185715, g_loss: 4.13032198
Epoch: [199] [  55/  60] time: 250.4200, d_loss: 0.04979715, g_loss: 5.15470648
Epoch: [199] [  56/  60] time: 250.9211, d_loss: 0.07891736, g_loss: 3.84598017
Epoch: [199] [  57/  60] time: 251.4152, d_loss: 0.04727333, g_loss: 4.55680847
Epoch: [199] [  58/  60] time: 251.9054, d_loss: 0.02132474, g_loss: 5.51441240
Epoch: [199] [  59/  60] time: 252.3839, d_loss: 0.03054039, g_loss: 5.26648092
[199/200] - ptime: 29.5673 loss_d: 0.04634481, loss_g: 4.74195242, lf:  0.00007748
Epoch: [200] [   0/  60] time: 270.1188, d_loss: 0.04388659, g_loss: 3.65519643
Epoch: [200] [   1/  60] time: 270.65

...

 [*] Finished generating Epoch: 106 , 100 new images!
INFO:tensorflow:Restoring parameters from checkpoint/Class2_Sample3_test1_1_1000_100_128_128/cDCGAN/cDCGAN.model-125


 98%|█████████▊| 59/60 [09:37<00:09,  9.78s/it]

 [*] Finished generating Epoch: 125 , 100 new images!
INFO:tensorflow:Restoring parameters from checkpoint/Class2_Sample3_test1_1_1000_100_128_128/cDCGAN/cDCGAN.model-143


100%|██████████| 60/60 [09:47<00:00,  9.79s/it]

 [*] Finished generating Epoch: 143 , 100 new images!
